## Import Keras and Packages

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.layers.convolutional import Conv2D # to add convolutional layers
from keras.layers.convolutional import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

## Download concrete data

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## A. Build a baseline model (5 marks)

## Build Neural Network Model

In [3]:
def regression_model(n_cols):
    # create model
    model = Sequential()
    
    # One hidden layer of 10 nodes, and a ReLU activation function
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    
    model.add(Dense(1))
    
    # Use the adam optimizer and the mean squared error as the loss function.
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [4]:
def trail_and_test(concrete_data, modeldef, n_epochs=50):
    # 1. Randomly split the data into a training and test sets by holding 30% of the data for testing
    train_data, test_data = train_test_split(concrete_data, test_size=0.3)

    ## Split data into predictors and target
    predictors_train = train_data[train_data.columns[train_data.columns != 'Strength']] # all columns except Strength
    target_train = train_data['Strength'] # Strength column

    predictors_test = test_data[test_data.columns[test_data.columns != 'Strength']] # all columns except Strength
    target_test = test_data['Strength'] # Strength column

    n_cols = predictors_train.shape[1] # number of predictors

    # 2. Train the model on the training data using 50 epochs.
    model = modeldef(n_cols=n_cols)
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=n_epochs, verbose=0)

    # 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.
    # You can use the mean_squared_error function from Scikit-learn.
    predictions = model.predict(predictors_test, batch_size=None, verbose=1, steps=None)
    mse = mean_squared_error(target_test, predictions)
    #model.evaluate(predictors_test, target_test, verbose=0)

    return mse

In [5]:
# 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
mses = np.array([trail_and_test(concrete_data, modeldef=regression_model, n_epochs=50) for i in range(50)])
# 5. Report the mean and the standard deviation of the mean squared errors
print('Mean: {mean}, Stddev: {std}'.format(mean=mses.mean(), std=mses.std()))

309/309 [==============================] - 1s 4ms/step
Mean: 539.2714937496307, Stddev: 660.4743586716176


## B. Normalize the data 

In [6]:
def train_and_test_normalised(concrete_data, modeldef, n_epochs):
    # 1. Randomly split the data into a training and test sets by holding 30% of the data for testing
    train_data, test_data = train_test_split(concrete_data, test_size=0.3)

    ## Split data into predictors and target
    predictors_train = train_data[train_data.columns[train_data.columns != 'Strength']] # all columns except Strength
    target_train = train_data['Strength'] # Strength column

    predictors_test = test_data[test_data.columns[test_data.columns != 'Strength']] # all columns except Strength
    target_test = test_data['Strength'] # Strength column

    # normalise predictors
    predictors_train_norm = (predictors_train - predictors_train.mean()) / predictors_train.std()
    predictors_test_norm = (predictors_test - predictors_test.mean()) / predictors_test.std()

    n_cols = predictors_train_norm.shape[1] # number of predictors

    # 2. Train the model on the training data using 50 epochs.
    model = modeldef(n_cols=n_cols)
    # Train the model on the training data using 50 epochs.
    model.fit(predictors_train_norm, target_train, validation_split=0.3, epochs=n_epochs, verbose=0)

    # 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.
    predictions = model.predict(predictors_test_norm, batch_size=None, verbose=1, steps=None)
    mse = mean_squared_error(target_test, predictions)
    print(mse)
    #model.evaluate(predictors_test, target_test, verbose=0)

    return mse

In [7]:
mses = np.array([train_and_test_normalised(concrete_data, modeldef=regression_model, n_epochs=50) for i in range(50)])
print('Mean: {mean}, Stddev: {std}'.format(mean=mses.mean(), std=mses.std()))

309/309 [==============================] - 1s 4ms/step
895.8366768559019
309/309 [==============================] - 1s 5ms/step
538.8349123503775
309/309 [==============================] - 1s 4ms/step
537.6522596509329
309/309 [==============================] - 1s 5ms/step
718.2132841960835
309/309 [==============================] - 2s 5ms/step
554.2272068624947
309/309 [==============================] - 1s 5ms/step
688.2612701336302
309/309 [==============================] - 1s 5ms/step
630.2078953375573
309/309 [==============================] - 2s 5ms/step
570.0388945656377
309/309 [==============================] - 2s 5ms/step
553.0614280542312
309/309 [==============================] - 2s 5ms/step
573.9188361694559
309/309 [==============================] - 2s 6ms/step
515.2981959631069
309/309 [==============================] - 2s 6ms/step
613.4661391813387
309/309 [==============================] - 2s 6ms/step
974.4034190455617
309/309 [==============================] - 2s 6ms/s

### How does the mean of the mean squared errors compare to that from Step A?

#### Step A(Baseline Model)
    - mean: 539.2714937496307
    - std: 660.4743586716176
#### Step B(Normalised feature):
    - mean: 677.2694909182007
    - std: 143.2904362488982

### Result
The mean of the MSE of B is a bit worse than A, but the standard deviation of B is much less than A

## C. Increate the number of epochs (5 marks)

In [8]:
# Repeat Part B but use 100 epochs this time for training.
mses = np.array([train_and_test_normalised(concrete_data, modeldef=regression_model, n_epochs=100) for i in range(50)])
print('Mean: {mean}, Stddev: {std}'.format(mean=mses.mean(), std=mses.std()))

309/309 [==============================] - 3s 10ms/step
223.85398452039425
309/309 [==============================] - 3s 10ms/step
517.0112666824906
309/309 [==============================] - 3s 10ms/step
273.9694154504999
309/309 [==============================] - 3s 10ms/step
192.82145966930696
309/309 [==============================] - 3s 10ms/step
210.51179032381697
309/309 [==============================] - 3s 10ms/step
203.76187841575143
309/309 [==============================] - 4s 12ms/step
275.7925823897395
309/309 [==============================] - 3s 11ms/step
248.05057504896035
309/309 [==============================] - 3s 11ms/step
197.05622773425705
309/309 [==============================] - 3s 11ms/step
406.27738068568686
309/309 [==============================] - 3s 11ms/step
225.62654009719517
309/309 [==============================] - 3s 11ms/step
266.88727204063923
309/309 [==============================] - 3s 11ms/step
217.397101004695
309/309 [=====================

### How does the mean of the mean squared errors compare to that from Step B?

#### Step B(Normalised feature):
    - mean: 670.1222133203134
    - std: 139.2909256592924
#### Step C(Train w/ double epoch):
    - mean: 228.31948220629343
    - std: 57.71126727954665

### Result
Both of the mean and the standard deviation of C is much better than B

## D. Increase the number of hidden layers (5 marks)

In [9]:
# - Three hidden layers, each of 10 nodes and ReLU activation function.
def regression_model_2(n_cols):
    # create model
    model = Sequential()
    
    # One hidden layer of 10 nodes, and a ReLU activation function
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    # Use the adam optimizer and the mean squared error as the loss function.
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [10]:
mses = np.array([train_and_test_normalised(concrete_data, modeldef=regression_model_2, n_epochs=50) for i in range(100)])
print('Mean: {mean}, Stddev: {std}'.format(mean=mses.mean(), std=mses.std()))

309/309 [==============================] - 5s 16ms/step
168.76705515794248
309/309 [==============================] - 5s 16ms/step
152.37805802615483
309/309 [==============================] - 5s 16ms/step
138.0862583685326
309/309 [==============================] - 5s 16ms/step
157.90373219208885
309/309 [==============================] - 5s 17ms/step
161.08597016054026
309/309 [==============================] - 5s 17ms/step
150.01149373859212
309/309 [==============================] - 5s 17ms/step
162.11951288517483
309/309 [==============================] - 5s 17ms/step
139.1288333203001
309/309 [==============================] - 5s 18ms/step
134.91886798127697
309/309 [==============================] - 5s 17ms/step
152.15621183209117
309/309 [==============================] - 6s 18ms/step
150.97036108938943
309/309 [==============================] - 5s 18ms/step
140.8121730021355
309/309 [==============================] - 6s 19ms/step
145.9784960636683
309/309 [====================

### How does the mean of the mean squared errors compare to that from Step B?

#### Step B(Normalised feature):
    - mean: 670.1222133203134
    - std: 139.2909256592924
#### Step D(Enhanced hidden layer):
    - mean: 148.9130410346697
    - std: 14.453772413834463

### Reuslt
Both of the mean and the standart deviation of MSE of D is significant better than B